<div style="text-align: center;">
    <h1> <b> ANÁLISIS DE RIESGO EN TIEMPO REAL DE REDES DE COMUNICACIONES INTERDEPENDIENTES AL SISTEMA ELÉCTRICO </b>
</div>

### **1. Importación de módulos necesarios**

In [ ]:
from pandapower                 import runpp
from matplotlib                 import pyplot as plt
from pandapower                 import networks as pn
from networkx.algorithms.flow   import maxflow
from pandapower                 import pandapowerNet
from seaborn                    import barplot
from scipy.interpolate          import griddata
from skimage                    import io
import plotly.graph_objects     as go
import numpy                    as np
import networkx                 as nx
import pandas                   as pd
import skimage.io               as sio
import random

Power Grid Model imported successfully


### **2. Funciones para inicializar la red**

In [ ]:
def calculate_transmission_line_capacity(line_info: dict, buses_info: dict, is_trafo: bool = False)-> float:
    """
    Función que calcula la capacidad
    """
    if is_trafo:
        capacity = line_info['sn_mva'] * line_info["max_loading_percent"] / 100
    else:
        EsEr = buses_info[line_info['from_bus']]['vn_kv']**2
        L = line_info['length_km']
        x = line_info['x_ohm_per_km']
        capacity = EsEr * np.sin(np.pi/6) / (L * x)
    return capacity

def create_graph(line_edges: list, trafo_edges: list, buses_info: dict, except_edges: list = []):
    """
    Función que crea un grafo de la red de potencia.
    """
    G = nx.Graph()
    for bus in buses_info:
        G.add_node(bus)
    for edge in line_edges:
        if edge not in except_edges:
            capacity = calculate_transmission_line_capacity(line_edges[edge], buses_info, is_trafo=False)
        else:
            capacity = 0
        G.add_edge(edge[0], edge[1], capacity=capacity)
    for edge in trafo_edges:
        if edge not in except_edges:
            capacity = calculate_transmission_line_capacity(trafo_edges[edge], buses_info, is_trafo=True)
        else:
            capacity = 0
        G.add_edge(edge[0], edge[1], capacity=capacity)
    return G

def degrees(G: nx.Graph):
    """
    Retorna un diccionario con los grados de los nodos.
    """
    degrees = []*len(G.nodes())
    for node in G.nodes():
        degrees.insert(node, G.degree(node))
    return degrees

### **3. Funciones para calcular las probabilidades de falla**
1. **Artículo** [3]: Power system cascading risk assessment based on complex network theory
2. **Sección** [2.2.1, 2.2.2, 2.2.3]: Transmission line overload, Bus power overload, Generator failure

In [3]:
def pij_line_overload_probability(line_id: int, net: pandapowerNet)->float:
    pij_avg = 0     # Average overload probability of line ij
    L = 0           # Line flow
    L_min = 0       # Minimum security setting
    L_max = 0       # Maximum security setting
    L_minlimit = 0  # Minimum limit of the line capacity
    L_maxlimit = 0  # Maximum limit of the line capacity
    
    if L_min <= L < L_max:
        probability = pij_avg
    elif L_max <= L < L_maxlimit:
        probability = ( ((1-pij_avg) * L) + (pij_avg * L_maxlimit) - (L_max) ) / (L_maxlimit - L_max)
    else:
        probability = 1
        
    return probability

def pf_frecuency_violation_probability(bus_id: int, net: pandapowerNet)->float:
    F_min = 0       # Minimum frequency margin
    F_max = 0       # Maximum frequency margin
    F_minlimit = 0  # Minimum frequency lower limit
    F_maxlimit = 0  # Maximum frequency upper limit
    pf_avg = 0      # Average frequency violation probability of generator i
    F = 0           # Generator frequency
    
    if F_min <= F < F_max:
        probability = pf_avg
    elif F_max <= F < F_maxlimit:
        probability = ( ((1-pf_avg) * F) + (pf_avg * F_maxlimit) - (F_max) ) / (F_maxlimit - F_max)
    elif F_minlimit <= F < F_min:
        probability = ( ((pf_avg-1) * F) + (F_min) - (pf_avg * F_minlimit) ) / (F_min - F_minlimit)
    else:
        probability = 1
        
    return probability

def pu_voltage_violation_probability(bus_id: int, net: pandapowerNet)->float:
    U_min = 0       # Minimum voltage margin
    U_max = 0       # Maximum voltage margin
    U_minlimit = 0  # Minimum voltage lower limit
    U_maxlimit = 0  # Maximum voltage upper limit
    pu_avg = 0      # Average voltage violation probability of generator i
    U = 0           # Generator voltage
    
    if U_min <= U < U_max:
        probability = pu_avg
    elif U_max <= U < U_maxlimit:
        probability = ( ((1-pu_avg) * U) + (pu_avg * U_maxlimit) - (U_max) ) / (U_maxlimit - U_max)
    elif U_minlimit <= U < U_min:
        probability = ( ((pu_avg-1) * U) + (U_min) - (pu_avg * U_minlimit) ) / (U_min - U_minlimit)
    else:
        probability = 1
        
    return probability

def pi_bus_power_overload_probability(bus_id: int, net: pandapowerNet)->float:
    pi_avg = 0      # Average over load probability of node i
    p = 0           # Total power injection for node i
    p_min = 0       # Minimum security setting
    p_max = 0       # Maximum security setting
    p_maxlimit = 0  # Maximum upper limit
    
    if p_min <= p < p_max:
        probability = pi_avg
    elif p_max <= p < p_maxlimit:
        probability = ( ((1-pi_avg) * p) + (pi_avg * p_maxlimit) - (p_max) ) / (p_maxlimit - p_max)
    else:
        probability = 1

    return probability

def is_generator(bus_id: int, net: pandapowerNet, generation)->bool:
    """
    Función que determina si un nodo es generador.
    """
    return bus_id in generation['bus'].values

def p_buses_failure_probability(bus_id: int, net: pandapowerNet, generation: pd.DataFrame)->float:
    """
    Función que calcula la probabilidad de que un nodo esté sobrecargado.
    """
    if is_generator(bus_id, net, generation):
        return max(pf_frecuency_violation_probability(bus_id, net), pu_voltage_violation_probability(bus_id, net))
    else:
        return pi_bus_power_overload_probability(bus_id, net)

### **4. Funciones para calcular el riesgo**
1. **Artículo** [3]: Power system cascading risk assessment based on complex network theory
2. **Sección** [2.2]: Risk assessment with improved CN model

In [ ]:
def bus_consequence(bus_id: int, net, array_deg: list):
    """
    Función que calcula la consecuencia de un bus específico en la red de potencia.
    """
    N1 = 0.5 # Proporción de la consecuencia topológica
    N2 = 0.5 # Proporción de la consecuencia eléctrica
    Ct = array_deg[bus_id] / max(array_deg) # Consecuencia topológica del bus i
    Ce = abs(net.res_bus.at[bus_id, 'p_mw']) / max(abs(net.res_bus['p_mw'])) # Consecuencia eléctrica del bus i
    Ci = N1 * Ct + N2 * Ce # Consecuencia del bus i
    return Ci, Ct, Ce

def line_consequence(line_id: tuple, m: list, n: list, graph: nx.Graph):
    """
    Función que calcula la consecuencia de una línea específica en la red de potencia.
    """
    sum_f_uv_ij = 0
    sum_f_uv_max = 0
    for u in m:
        for v in n:
            if u != v:
                flows = maxflow.maximum_flow(graph, u, v)
                f_uv_ij = flows[1][line_id[0]][line_id[1]]
                f_uv_max = flows[0]
                sum_f_uv_ij += f_uv_ij
                sum_f_uv_max += f_uv_max
    return sum_f_uv_ij / sum_f_uv_max

def calculate_buses_risk(buses_info: dict, net: pandapowerNet, array_deg: list, generation: pd.DataFrame)->tuple:
    """
    Función que calcula el riesgo de cada nodo de la red de potencia.
    """
    risks = []
    consequences = []
    topological_consequences = []
    electrical_consequences = []
    for bus_id in buses_info:
        consequence, topological_consequence, electrical_consequence = bus_consequence(bus_id, net, array_deg)
        risk = consequence * p_buses_failure_probability(bus_id, net, generation)
        risks.append(risk)
        consequences.append(consequence)
        topological_consequences.append(topological_consequence)
        electrical_consequences.append(electrical_consequence)
    return np.array(risks), np.array(consequences), np.array(topological_consequences), np.array(electrical_consequences)

def calculate_lines_risk(m: list, n: list, graph: nx.Graph, net: pandapowerNet, line_edges: dict, trafo_edges: dict)->dict:
    """
    Función que calcula el riesgo de cada línea de la red de potencia.
    """
    risks = {}
    consequences = {}
    for line in list(line_edges.keys()) + list(trafo_edges.keys()):
        consequence = line_consequence(line, m, n, graph)
        risk = consequence * pij_line_overload_probability(line, net)
        risks[line] = risk
        consequences[line] = line_consequence(line, m, n, graph)
    return risks, consequences
        

### **5. Funciones para simular el flujo de un evento en cascada**
1. **Artículo** [3]: Power system cascading risk assessment based on complex network theory
2. **Sección** [3]: Cascading event simulation module

| Variable       | Tipo       | Descripción                                                                 |
|----------------|------------|-----------------------------------------------------------------------------|
| net            | `pandapowerNet` | Red de potencia creada con la librería PandaPower para el caso seleccionado |
| source_nodes   | `list`     | Lista de los buses dentro del sistema que hacen parte de la generación      |
| sink_nodes     | `list`     | Lista de los buses dentro del sistema que hacen parte de la carga           |
| line_edges     | `dict`     | Diccionario con las líneas de transmisión y sus propiedades                 |
| trafo_edges    | `dict`     | Diccionario con los transformadores y sus propiedades                      |
| buses_info     | `dict`     | Diccionario con la información de los buses                                 |
| graph          | `Graph`    | Grafo de la red de potencia del caso de estudio                             |
| array_deg      | `list`     | Lista con los grados de los nodos del grafo                                 |
| generation      | `DataFrame` | Información de los generadores dentro del sistema                              |
| external_grid   | `DataFrame` | Información de la red externa conectada al sistema                             |
| loads           | `DataFrame` | Información de las cargas dentro del sistema                                   |

In [5]:
def data_input(case: str):
    """
    Función que recoge los datos de entrada para la simulación de eventos de cascada.
    """
    net             = pn.__getattribute__(case)()
    source_nodes    = net.gen.bus.to_list()
    sink_nodes      = net.load.bus.to_list()
    line_edges      = {(net.line.from_bus.at[i], net.line.to_bus.at[i]): net.line.loc[i].to_dict() for i in net.line.index}
    trafo_edges     = {(net.trafo.hv_bus.at[i], net.trafo.lv_bus.at[i]): net.trafo.loc[i].to_dict() for i in net.trafo.index}
    buses_info      = {bus: net.bus.loc[bus].to_dict() for bus in net.bus.index}
    graph           = create_graph(line_edges, trafo_edges, buses_info)
    array_deg       = degrees(graph)
    generation      = net.gen
    return net, source_nodes, sink_nodes, line_edges, trafo_edges, buses_info, graph, array_deg, generation
    

def steady_state_distribution(net: pandapowerNet):
    """
    Función que calcula la distribución de estado estable de la red de potencia.
    """
    runpp(net,init="results", enforce_q_lims=True)

def n_1_contingency(net: pandapowerNet, id: int | tuple, type: str):
    """
    Función que simula una contingencia N-1 en la red de potencia.
    """
    if type == 'line':
        from_bus = id[0]
        to_bus = id[1]
        net.line.loc[(net.line.from_bus == from_bus) & (net.line.to_bus == to_bus), 'in_service'] = False
        net.trafo.loc[(net.trafo.hv_bus == from_bus) & (net.trafo.lv_bus == to_bus), 'in_service'] = False
    elif type == 'trafo':
        net.trafo.at[id, 'in_service'] = False
    else:
        net.bus.at[id, 'in_service'] = False
    return net
    

def get_new_operating_point(net: pandapowerNet, lines_triped: list):
    """
    Función que calcula el nuevo punto de operación de la red de potencia.
    """
    source_nodes    = net.gen.bus.to_list()
    sink_nodes      = net.load.bus.to_list()
    line_edges      = {(net.line.from_bus.at[i], net.line.to_bus.at[i]): net.line.loc[i].to_dict() for i in net.line.index}
    trafo_edges     = {(net.trafo.hv_bus.at[i], net.trafo.lv_bus.at[i]): net.trafo.loc[i].to_dict() for i in net.trafo.index}
    buses_info      = {bus: net.bus.loc[bus].to_dict() for bus in net.bus.index}
    graph           = create_graph(line_edges, trafo_edges, buses_info, except_edges=lines_triped)
    array_deg       = degrees(graph)
    generation      = net.gen
    try:
        runpp(net,init="results", enforce_q_lims=True)
    except:
        print("Power flow did not converge")
        net.converged = False
    
    return net, source_nodes, sink_nodes, line_edges, trafo_edges, buses_info, graph, array_deg, generation
    


def risk_assessment(buses_info: dict, net: pandapowerNet, array_deg: list, generation: pd.DataFrame, source_nodes: list, sink_nodes: list, graph: nx.Graph, line_edges: dict, trafo_edges: dict):
    """
    Función que evalúa el riesgo de la red de potencia.
    """
    risks, consequences, topological_consequences, electrical_consequences = calculate_buses_risk(buses_info, net, array_deg, generation)
    line_risks, line_consequences = calculate_lines_risk(source_nodes, sink_nodes, graph, net, line_edges, trafo_edges)
    return risks, consequences, topological_consequences, electrical_consequences, line_risks, line_consequences

def rank_risks(line_risks: np.array):
    """
    Función que clasifica los riesgos de la red de potencia.
    """
    rank = sorted(line_risks.items(), key=lambda x: x[1], reverse=True)
    return rank

def trip_edge_or_node_with_highest_ranking(net: pandapowerNet, rank: np.array):
    """
    Función que desconecta el nodo o la línea con mayor riesgo.
    """
    return n_1_contingency(net, rank[0][0], 'line')

def does_power_flow_converge(net: pandapowerNet)->bool:
    """
    Función que determina si el flujo de potencia converge.
    """
    return net.converged
    
    
def calculate_total_shedding():
    """
    Función que calcula la cantidad total de carga que se ha perdido.
    """
    pass


def is_system_overloaded(net: pandapowerNet) -> bool:
    """
    Función que determina si el sistema está sobrecargado.
    """
    for line in net.res_line.index:
        if net.res_line.at[line, 'p_from_mw'] != 0 or net.res_line.at[line, 'p_to_mw'] != 0:
            return False
    print("System Overloaded")
    return True

def cascading_event_simulation(case: str):
    """
    Función que simula un evento de cascada en la red de potencia.
    """
    lines_triped = []
    risks_hist = []
    consequences_hist = []
    topological_consequences_hist = []
    electrical_consequences_hist = []
    line_risks_hist = []
    line_consequences_hist = []
    i = 0 
    net, source_nodes, sink_nodes, line_edges, trafo_edges, buses_info, graph, array_deg, generation = data_input(case)
    steady_state_distribution(net)
    while True:
        risks, consequences, topological_consequences, electrical_consequences, line_risks, line_consequences = risk_assessment(buses_info, net, array_deg, generation, source_nodes, sink_nodes, graph, line_edges, trafo_edges)
        
        # Update history
        risks_hist.append(risks)
        consequences_hist.append(consequences)
        topological_consequences_hist.append(topological_consequences)
        electrical_consequences_hist.append(electrical_consequences)
        line_risks_hist.append(line_risks)
        line_consequences_hist.append(line_consequences)

        line_risks = rank_risks(line_risks)
        lines_triped.append(line_risks[0][0])

        net = trip_edge_or_node_with_highest_ranking(net, line_risks)
        print(f"Line triped at stage {i}: {(line_risks[0][0][0]+1,line_risks[0][0][1]+1)}")
        net, source_nodes, sink_nodes, line_edges, trafo_edges, buses_info, graph, array_deg, generation = get_new_operating_point(net, lines_triped)
        if does_power_flow_converge(net) == False or is_system_overloaded(net) == True:
            return risks_hist, consequences_hist, topological_consequences_hist, electrical_consequences_hist, line_risks_hist, line_consequences_hist, lines_triped, net
        i += 1

### **6. Funciones para graficar las simulaciones**

In [ ]:
def plot_results(results: tuple,  case: str, lim: int = 3):
    """
    Función que genera los gráficos de la red de potencia.
    """
    risks_lst, consequences_lst, topological_consequences_lst, electrical_consequences_lst, line_risks_lst, line_consequences_lst, lines_triped, net = results
    
    # Gráfica 1: Stacked Buses Consequences
    fig, ax_bus_consequences = plt.subplots(figsize=(15, 7))
    data = []
    if len(consequences_lst) < lim:
        rango = range(len(consequences_lst))
    else:
        rango = range(lim)
        
    for i in rango:
        stage = f"Stage {i}"
        consequences = consequences_lst[i]
        topological_consequences = topological_consequences_lst[i]
        electrical_consequences = electrical_consequences_lst[i]
        topological_consequences = topological_consequences / np.linalg.norm(consequences)
        electrical_consequences = electrical_consequences / np.linalg.norm(consequences)
        for node in range(len(consequences)):
            data.append({
                "Node": node + 1,
                "Stage": stage,
                "Topological": topological_consequences[node],
                "Electrical": electrical_consequences[node]
            })
    df = pd.DataFrame(data)
    nodes = df["Node"].unique()
    width = 1/len(df["Stage"].unique())
    x = np.arange(len(nodes))

    for i in rango:
        stage = f"Stage {i}"
        stage_data = df[df["Stage"] == stage]
        topological = stage_data["Topological"]
        electrical = stage_data["Electrical"]
        ax_bus_consequences.bar(x + i * width, topological, width-0.1, label=f"{stage} - Topological")
        ax_bus_consequences.bar(x + i * width, electrical, width-0.1, bottom=topological, label=f"{stage} - Electrical")
    ax_bus_consequences.set_xticks(x + width * (len(df["Stage"].unique()) - 1) / 2)
    ax_bus_consequences.set_xticklabels(nodes)
    ax_bus_consequences.set_xlabel("Nodes")
    ax_bus_consequences.set_ylabel("Consequences (Normalized)")
    ax_bus_consequences.set_title("Node Consequences by Stage")
    ax_bus_consequences.legend(title="Consequences")
    plt.tight_layout()
    
    # Gráfica 2: Line Consequences
    data = []
    for i in rango:
        line_consequences = line_consequences_lst[i]
        stage = f"Stage {i}"
        for line, value in line_consequences.items():
            data.append({"Line": str((line[0] + 1, line[1] + 1)), "Consequence": value, "Stage": stage})
    df = pd.DataFrame(data)
    
    plt.figure(figsize=(15, 7))
    barplot(
        data=df, 
        x="Line", 
        y="Consequence", 
        hue="Stage", 
        palette="Set2",
        gap=0.1
    )
    plt.xlabel("Línea")
    plt.ylabel("Consecuencia")
    plt.title("Consecuencias de las Líneas de la Red de Potencia")
    plt.xticks(rotation=90)
    plt.legend(title="Stage")
    plt.tight_layout()
    
    # Gráfica 3: Buses Risk
    plt.figure(figsize=(15, 7))  
    data = []
    for i in rango:
        risks = risks_lst[i]
        stage = f"Stage {i}"
        for node, risk in enumerate(risks, start=1):
            data.append({"Node": node, "Risk": risk, "Stage": stage})

    df = pd.DataFrame(data)
    barplot(
        data=df, 
        x="Node", 
        y="Risk", 
        hue="Stage", 
        palette="Set2",
        gap=0.1
    )
    plt.xlabel("Nodo")
    plt.ylabel("Riesgo")
    plt.title("Riesgos de los Nodos de la Red de Potencia")
    plt.xticks(range(len(risks_lst[0])), [str(i + 1) for i in range(len(risks_lst[0]))])
    plt.legend(title="Stage", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
        
    # Gráfica 4: Lines Risk
    plt.figure(figsize=(15, 7))
    data = []
    for i in rango:
        line_risks = line_risks_lst[i]
        stage = f"Stage {i}"
        for line, risk in line_risks.items():
            data.append({"Line": str((line[0] + 1, line[1] + 1)), "Risk": risk, "Stage": stage})
    df = pd.DataFrame(data)
    barplot(
        data=df, 
        x="Line", 
        y="Risk", 
        hue="Stage", 
        palette="Set2",
        gap=0.1
    )
    plt.xlabel("Línea")
    plt.ylabel("Riesgo")
    plt.title("Riesgos de las Líneas de la Red de Potencia")
    plt.xticks(rotation=90)
    plt.legend(title="Stage", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    
    # Save plots
    fig.savefig(f"Results/{case}/bus_consequences.png")
    plt.savefig(f"Results/{case}/line_consequences.png")
    plt.savefig(f"Results/{case}/bus_risks.png")
    plt.savefig(f"Results/{case}/line_risks.png")
    
def plot_geographical_distribution(results: tuple, stage: int, case: str, interpol: str):
    risks_lst, consequences_lst, topological_consequences_lst, electrical_consequences_lst, line_risks_lst, line_consequences_lst, lines_triped, net = results
    
    buses_risk = risks_lst[stage]
    lines_risk = line_risks_lst[stage]
    
    buses_coords = net.bus_geodata.copy()
    buses_coords['risk'] = buses_risk
    buses_coords['id'] = buses_coords.index + 1
    del buses_coords["coords"]
    lines_coords = pd.DataFrame()
    for start, end in lines_risk:
        if start in buses_coords.index and end in buses_coords.index:
            coord = ((buses_coords.loc[start, 'x'] + buses_coords.loc[end, 'x']) / 2, 
                     (buses_coords.loc[start, 'y'] + buses_coords.loc[end, 'y']) / 2)
            lines_coords = pd.concat([lines_coords, pd.DataFrame([{'x': coord[0], 'y': coord[1], 'id': (start, end), 'risk': lines_risk[(start, end)]}])], ignore_index=True)
    
    coordinates = pd.concat([buses_coords, lines_coords])

    
    # Crear una malla para interpolar los datos
    x = np.array(coordinates.x)
    y = np.array(coordinates.y)
    z = np.array(coordinates.risk)

    xi = np.linspace(min(x), max(x), 1000) 
    yi = np.linspace(min(y), max(y), 800)
    xi, yi = np.meshgrid(xi, yi)
    
    # Interpolación cúbica (spline)
    
    zi = griddata((x, y), z, (xi, yi), method=interpol, rescale=True)

    image = sio.imread ("Results/"+case+"/graph.png", as_gray=True)
    io.imshow(image)


    fig = go.Figure(data=[go.Surface(z=zi, x=xi, y=yi, colorscale='Jet')])
    
    fig.add_surface(x=xi, y=yi, z=np.ones(zi.shape),
                    surfacecolor=image,
                    colorscale='gray',
                    showscale=False)         

    fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))

    fig.update_layout(title=dict(text='Steady state risk geographical distribution'),
                      width=800, height=1000,
                      autosize = True)
    # fig.show()
    fig.write_image(f"Results/{case}/geographical_distribution_stage_{stage}.png")
    
    
    
    
    
    

<div style="text-align: center;">
    <h2> <b> CASO DE ESTUDIO IEEE 14 BUSES</b>
</div>

In [21]:
case = "case14"
results = cascading_event_simulation(case)

Line triped at stage 0: (7, 9)
Line triped at stage 1: (4, 9)
Line triped at stage 2: (4, 5)
Line triped at stage 3: (5, 6)
Line triped at stage 4: (6, 13)
Line triped at stage 5: (6, 11)
Line triped at stage 6: (1, 5)
Line triped at stage 7: (3, 4)
Line triped at stage 8: (6, 12)
Line triped at stage 9: (2, 5)
Line triped at stage 10: (2, 4)
Line triped at stage 11: (2, 3)
Line triped at stage 12: (1, 2)
System Overloaded


In [ ]:
# plot_results(results, case, lim=3)

In [ ]:
# plot_geographical_distribution(results, 1, case, "cubic")

In [14]:
case = "case39"
results = cascading_event_simulation(case)

Line triped at stage 0: (16, 17)
Line triped at stage 1: (2, 3)
Line triped at stage 2: (26, 27)
Power flow did not converge


In [ ]:
# plot_results(results, case, 3)

In [ ]:
# plot_geographical_distribution(results, 1, case, "cubic")

In [ ]:
def generateScaleFreeComsGraph(n, n0):
    # create initial graph with n0 nodes for MultiDiGraph
    initial_graph = nx.multidigraph.MultiDiGraph()
    initial_graph.add_nodes_from(list(range(n0)))
    # connect scale-free graph with n-n0 nodes
    graph = nx.generators.scale_free_graph(n)
    return graph

def generateRandomComsGraph(n,p):
    graph = nx.erdos_renyi_graph(n, p)
    return graph

def generateRandomConnectionsBetweenGraphs(graph_1, graph_2):
    graph_1_nodes = list(graph_1.nodes())
    graph_2_nodes = list(graph_2.nodes())
    graph_1_nodes.sort()
    random_connetions = []*len(graph_1_nodes)
    for node in graph_1_nodes:
        random_connetions.insert(node, random.choice(graph_2_nodes))
        graph_2_nodes.remove(random_connetions[node])
    return random_connetions

def generateD2DConnectionsBetweenGraphs(graph_1, graph_2):
    graph_1_degrees = list(graph_1.degree())
    graph_2_degrees = list(graph_2.degree())
    graph_1_degrees.sort(key=lambda x: x[1], reverse=True)
    graph_2_degrees.sort(key=lambda x: x[1], reverse=True)
    d2d_connetions = []*len(graph_1_degrees)
    i = 0
    for node in graph_1_degrees:
        d2d_connetions.insert(node[0], graph_2_degrees[i][0])
        i += 1
    return d2d_connetions

def number_of_packets(node):
    pass

def calculate_sum_Li(neighbors):
    pass

def pj_chosen_probability(i, neighbors):
    sum_Li = calculate_sum_Li(neighbors)
    for j, neighbor in enumerate(neighbors):
        Beta = 20                      # Inverse of the tempreature of the coms network
        hd = 0.75 
        dj = p[j][i]                # length of the shortest path between node j and the destination node
        cj = number_of_packets(j)   # number of message packets in the queues of node i

        Hj = hd*dj + (1-hd)*cj      # hd and β are constants
        Pj  = np.exp(-Beta*Hj) / sum_Li
        
    return Pj

def tij_inverse_time_overcurrent_protection(Iij: float, I_set_ij: float):
    K = 7                       
    alpha = 0.3 
    tij = K / ( abs(Iij/I_set_ij)**alpha - 1)
    return tij

In [17]:
# Potencia generada total (P y Q)}
# print(net)
# total_generation = net.res_gen['p_mw'].sum() + net.res_ext_grid['p_mw'].sum()  # Activa
# total_load = net.load['p_mw'].sum()          # Activa
# total_losses = net.res_line['pl_mw'].sum()   # Pérdidas
# print(f"Generación: {total_generation} MW, Carga: {total_load} MW, Pérdidas: {total_losses} MW")

# # Identificar buses con tensiones fuera del rango
# out_of_limits = net.res_bus[(net.res_bus.vm_pu < 0.94) | (net.res_bus.vm_pu > 1.06)]
# print("Buses fuera de límites:")
# print(out_of_limits)

# # Identificar líneas sobrecargadas
# overloaded_lines = net.res_line[net.res_line.loading_percent > 100]
# print("Líneas sobrecargadas:")
# print(overloaded_lines)